<a href="https://colab.research.google.com/github/theKirill/GensimSample/blob/master/Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gensim
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Activation
from keras.models import Model
import sklearn
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import save_model, load_model
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import RegexpTokenizer

Using TensorFlow backend.


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
file_csv = pd.read_csv('drive/My Drive/ForGensim/train.csv')

In [0]:
file_csv

,text,intents
0,Add another song to the Cita RomГЎntica playli...,AddToPlaylist
1,add clem burke in my playlist Pre-Party R&B Jams,AddToPlaylist
2,Add Live from Aragon Ballroom to Trapeo,AddToPlaylist
3,add Unite and Win to my night out,AddToPlaylist
4,Add track to my Digster Future Hits,AddToPlaylist
...,...,...
11369,Is Across the Line playing at the closest movi...,SearchScreeningEvent
11370,Which animated movies are playing in the neigh...,SearchScreeningEvent
11371,Where is They Always Return at Dawn playing,SearchScreeningEvent
11372,What is the movie schedule in the neighborhood,SearchScreeningEvent


In [0]:
all_texts = file_csv['text']
all_intents = file_csv['intents']

In [0]:
file_csv['text']

0        Add another song to the Cita RomГЎntica playli...
1         add clem burke in my playlist Pre-Party R&B Jams
2                  Add Live from Aragon Ballroom to Trapeo
3                        add Unite and Win to my night out
4                      Add track to my Digster Future Hits
                               ...                        
11369    Is Across the Line playing at the closest movi...
11370    Which animated movies are playing in the neigh...
11371          Where is They Always Return at Dawn playing
11372       What is the movie schedule in the neighborhood
11373    Tell me when Howling II: Your Sister Is a Were...
Name: text, Length: 11374, dtype: object

In [0]:
unique_intents = file_csv.intents.unique()

In [8]:
unique_intents

array(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic',
       'SearchScreeningEvent'], dtype=object)

**Удаление стоп-слов и пунктуации**

In [0]:
def remove_stopwords(text):
  regex_tokenizer = RegexpTokenizer('[a-zA-Z]+')
  words = regex_tokenizer.tokenize(text.lower())
  stop_words = set(stopwords.words("english"))
  without_stop_words = [w for w in words if w not in stop_words]
  return without_stop_words

In [0]:
def get_tokens(sentences):
  tokens = []

  for sentence in sentences:
    tokens.append(sentence)
    
  return tokens

In [0]:
sentences = np.array(all_texts.apply(lambda x : remove_stopwords(x)))

In [0]:
tokens = get_tokens(sentences)

In [0]:
tokens[:5]

[['add', 'another', 'song', 'cita', 'rom', 'ntica', 'playlist'],
 ['add', 'clem', 'burke', 'playlist', 'pre', 'party', 'r', 'b', 'jams'],
 ['add', 'live', 'aragon', 'ballroom', 'trapeo'],
 ['add', 'unite', 'win', 'night'],
 ['add', 'track', 'digster', 'future', 'hits']]

**Работа с Word2Vec**

In [0]:
VECTOR_SIZE = 200
WINDOW_SIZE = 10
MIN_COUNT = 3
ITER_COUNT = 10
WORKERS = 4

In [0]:
word2vec_model = Word2Vec(tokens, size = VECTOR_SIZE, window = WINDOW_SIZE, min_count = MIN_COUNT, workers = WORKERS, iter = ITER_COUNT)

In [0]:
#word2vec_model.train(tokens, total_examples = len(tokens), epochs = 500)

(22181828, 34511000)

In [0]:
word2vec_model['song']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.0437184 , -0.0073642 , -0.44962013,  0.27555406, -0.24916933,
       -0.07646467,  0.09827101,  0.3209451 , -0.30768815,  0.00835073,
        0.32874823, -0.19451049,  0.21499723, -0.21461548, -0.00085246,
       -0.01956361,  0.5232777 ,  0.25570333, -0.40360826, -0.19600329,
        0.08707048,  0.26422572, -0.15037163, -0.45917276, -0.0627289 ,
       -0.10912298,  0.11421495,  0.20208748, -0.4818529 , -0.44779626,
        0.08362344, -0.5636349 ,  0.3081727 , -0.07151476,  0.21944045,
       -0.11744327, -0.16783795,  0.03537119,  0.13302346, -0.12718078,
        0.3389973 , -0.12286204,  0.29145148,  0.13536686,  0.13336745,
        0.3457266 , -0.01270729, -0.07166635, -0.21400295, -0.7249681 ,
       -0.04706166,  0.6681783 , -0.21374068,  0.32801357, -0.50251675,
       -0.18216309,  0.46159118, -0.29116812, -0.7497172 , -0.42112634,
        0.07191984,  0.7347315 , -0.21682791, -0.18609563, -0.5109359 ,
       -0.19564581, -0.23504876, -0.2627492 ,  0.13472463,  0.02

In [0]:
word2vec_model.most_similar('song')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('track', 0.9973636269569397),
 ('album', 0.997258186340332),
 ('artist', 0.9963237047195435),
 ('tune', 0.9955297708511353),
 ('put', 0.99542635679245),
 ('rock', 0.9944835901260376),
 ('skies', 0.9935083389282227),
 ('playlist', 0.9932403564453125),
 ('music', 0.9932096004486084),
 ('google', 0.993181049823761)]

In [0]:
word2vec_model.most_similar(positive=['song'], negative=['artist'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('aired', 0.21036595106124878),
 ('list', 0.168114572763443),
 ('b', 0.15644524991512299),
 ('play', 0.12908120453357697),
 ('ines', 0.11604739725589752),
 ('playing', 0.11349107325077057),
 ('love', 0.0959177166223526),
 ('nearest', 0.08513445407152176),
 ('theatres', 0.08001292496919632),
 ('see', 0.07834508270025253)]

In [15]:
word2vec_model.save('drive/My Drive/ForGensim/Word2VecModel')
word2vec_model.wv.save_word2vec_format('drive/My Drive/ForGensim/Word2VecModel.txt', binary=False)
word2vec_model.wv.save_word2vec_format('drive/My Drive/ForGensim/Word2VecModel.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Работа с keras**

In [0]:
def transform_from_categorical(prediction, intents):
  answers = []

  for i in range(prediction.shape[0]):
    answers.append(intents[prediction[i]])

  return answers

In [0]:
def transform_to_categorical(intents_for_each, unique_intents):
  intents_to_digit = []

  for intent in intents_for_each:
    intents_to_digit.append(unique_intents.index(intent))

  return to_categorical(intents_to_digit, len(unique_intents))

In [0]:
# Создание модели
def get_model(word2vec_model, COUNT_CLASSES):
    model = Sequential() 
    model.add(word2vec_model.wv.get_keras_embedding(train_embeddings = True))
    
    model.add(Dropout(0.2))

    model.add(Conv1D(50,
                 3,
                 padding = 'valid',
                 activation = 'relu',
                 strides = 1))
    model.add(GlobalMaxPooling1D())

    model.add(Dense(250))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))

    model.add(Dense(COUNT_CLASSES, activation = 'softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
  
    return model

In [0]:
word_vectors = word2vec_model.wv

In [0]:
print("Number of word vectors: {}".format(len(word_vectors.vocab)))

Number of word vectors: 2400


In [0]:
texts = sentences

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
text = tokenizer.texts_to_sequences(texts)
text = pad_sequences(text, maxlen = 10)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(text, all_intents, test_size = 0.2, stratify = all_intents)

In [0]:
y_train = transform_to_categorical(y_train, unique_intents.tolist())

In [0]:
y_test =  transform_to_categorical(y_test, unique_intents.tolist())

In [0]:
COUNT_CLASSES = unique_intents.shape[0]

In [26]:
keras_model = get_model(word2vec_model, COUNT_CLASSES)
keras_model.summary()











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         480000    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 50)          30050     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               12750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0   

In [0]:
BATCH = 32
EPOCHS_COUNT = 10

In [28]:
keras_model.fit(X_train, y_train, batch_size = BATCH, epochs = EPOCHS_COUNT, validation_data = (X_test, y_test),
             callbacks=[ModelCheckpoint('drive/My Drive/ForGensim/keras_model.h5', save_best_only = True)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 9099 samples, validate on 2275 samples
Epoch 1/10
9099/9099 [==============================] - 6s 692us/step - loss: 0.5065 - acc: 0.8153 - val_loss: 0.0660 - val_acc: 0.9859
Epoch 2/10
9099/9099 [==============================] - 2s 256us/step - loss: 0.0657 - acc: 0.9833 - val_loss: 0.0303 - val_acc: 0.9921
Epoch 3/10
9099/9099 [==============================] - 2s 266us/step - loss: 0.0350 - acc: 0.9916 - val_loss: 0.0225 - val_acc: 0.9943
Epoch 4/10
9099/9099 [==============================] - 2s 259us/step - loss: 0.0175 - acc: 0.9956 - val_loss: 0.0236 - val_acc: 0.9943
Epoch 5/10
9099/9099 [==============================] - 2s 265us/step - loss: 0.0137 - acc: 0.9959 - val_loss: 0.0204 - val_acc: 0.9947
Epoch 6/10
9099/9099 [==============================] - 2s 256us/step - loss: 0.0111 - acc: 0.9969 - val_loss: 0.0264 - val_acc: 0.9934
Epoch 7/10
9099/9099 [==================

In [0]:
test_data=pd.read_csv('drive/My Drive/ForGensim/test.csv')

In [30]:
test_data

,text
0,Add another song to my playlist
1,get weather tomorrow
2,what is the weather today?
3,turn on my music
4,what is the temperature this night in Kirov?


In [0]:
#переводим тестовые предложения в вектора
test_texts = np.array(test_data.text.apply(lambda x : remove_stopwords(x)))
sequences_test = tokenizer.texts_to_sequences(test_texts)
X_predict = pad_sequences(sequences_test, maxlen = X_train.shape[1])

In [0]:
prediction = keras_model.predict_classes(X_predict)

In [34]:
answers = transform_from_categorical(prediction, unique_intents.tolist())
answers

['AddToPlaylist', 'GetWeather', 'GetWeather', 'PlayMusic', 'GetWeather']

**Дообучение модели**

In [0]:
def add_new_intents(old, new):
  result = old

  for i in new:
    result.append(i)

  return result

In [0]:
new_file_csv = pd.read_csv('drive/My Drive/ForGensim/train_new.csv')

In [0]:
all_new_texts = new_file_csv['text']
all_new_intents = new_file_csv['intents']

In [0]:
unique_new_intents = new_file_csv.intents.unique()

In [308]:
unique_new_intents

array(['RateBook', 'SearchCreativeWork'], dtype=object)

In [0]:
all_unique_intents = add_new_intents(unique_intents.tolist(), unique_new_intents.tolist())

In [0]:
tokens_new = get_tokens(all_new_texts)

In [0]:
tokens_new[:5]

[['rate', 'lotus', 'storm', 'zero'],
 ['rate', 'fall', 'artist', 'stars'],
 ['rate', 'current', 'novel', 'one', 'points'],
 ['rate', 'ape', 'man', 'within'],
 ['give', 'penalty', 'three', 'stars']]

In [0]:
texts_new = np.array(all_new_texts.apply(lambda x : remove_stopwords(x)))

In [312]:
word2vec_model = Word2Vec.load("drive/My Drive/ForGensim/Word2VecModel")

INFO:gensim.utils:loading Word2Vec object from drive/My Drive/ForGensim/Word2VecModel
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
INFO:gensim.utils:loading wv recursively from drive/My Drive/ForGensim/Word2VecModel.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute vectors_norm to None
INFO:gensim.utils:loading vocabulary recursively from drive/My Drive/ForGensim/Word2VecModel.vocabulary.* with mmap=None
INFO:gensim.utils:loading trainables recursively from drive/My Drive/ForGensim/Word2VecModel.trainables.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:loaded drive/My Drive/ForGensim/Word2VecModel


In [313]:
#дообучаем модель word2vec
word2vec_model.build_vocab(tokens_new, update = True)
word2vec_model.train(tokens_new, total_examples = len(tokens_new), epochs = word2vec_model.iter)

INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:collected 94 word types from a corpus of 185198 raw words and 4510 sentences
INFO:gensim.models.word2vec:Updating model with new vocabulary
INFO:gensim.models.word2vec:New added 79 unique words (45% of original 173) and increased the count of 79 pre-existing words (45% of original 173)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 94 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 76 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 86394 word corpus (46.7% of prior 185179)
INFO:gensim.models.base_any2vec:estimated required memory for 158 words and 200 dimensions: 331800 bytes
INFO:gensim.models.word2vec:updating layer weights
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute

(432186, 1851980)

In [0]:
keras_model = load_model('drive/My Drive/ForGensim/keras_model.h5')

In [0]:
COUNT_NEW_CLASSES = len(all_unique_intents)

In [0]:
#запоминаем все слои
layers = []
for i in range(len(all_unique_intents)):
  layers.append(keras_model.layers.pop())
  keras_model.pop()

keras_model.pop()

In [0]:
#меняем первый слой с новой обученной модель word2vec
keras_model = Sequential()
keras_model.add(word2vec_model.wv.get_keras_embedding(train_embeddings = True))

In [0]:
new_layers_count = len(layers) - 1
for i in range(new_layers_count):
  keras_model.add(layers[new_layers_count - i])

In [0]:
keras_model.add(Dense(COUNT_NEW_CLASSES, activation = 'softmax'))

In [0]:
keras_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [325]:
keras_model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, None, 200)         493000    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 50)          30050     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               12750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)             

In [0]:
kkk=texts.tolist()

for i in range(len(texts_new)):
  kkk.append(texts_new[i])

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(kkk)
text = tokenizer.texts_to_sequences(kkk)
text = pad_sequences(text, maxlen = 10)

In [0]:
all_intents_all=all_intents.tolist()
for i in range(len(all_new_intents)):
  all_intents_all.append(all_new_intents[i])

In [0]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(text, all_intents_all, test_size=0.2, stratify=all_intents_all)

In [350]:
all_unique_intents

['AddToPlaylist',
 'BookRestaurant',
 'GetWeather',
 'PlayMusic',
 'SearchScreeningEvent',
 'RateBook',
 'SearchCreativeWork']

In [0]:
y_train_new = transform_to_categorical(y_train_new, all_unique_intents)

In [0]:
y_test_new =  transform_to_categorical(y_test_new, all_unique_intents)

In [353]:
keras_model.fit(X_train_new, y_train_new, batch_size=BATCH, epochs=EPOCHS_COUNT, validation_data=(X_test_new, y_test_new))

Train on 12707 samples, validate on 3177 samples
Epoch 1/10
12707/12707 [==============================] - 9s 687us/step - loss: 0.5221 - acc: 0.8272 - val_loss: 0.1119 - val_acc: 0.9710
Epoch 2/10
12707/12707 [==============================] - 4s 293us/step - loss: 0.1004 - acc: 0.9708 - val_loss: 0.0843 - val_acc: 0.9773
Epoch 3/10
12707/12707 [==============================] - 4s 298us/step - loss: 0.0620 - acc: 0.9810 - val_loss: 0.0809 - val_acc: 0.9773
Epoch 4/10
12707/12707 [==============================] - 4s 292us/step - loss: 0.0411 - acc: 0.9865 - val_loss: 0.0784 - val_acc: 0.9732
Epoch 5/10
12707/12707 [==============================] - 4s 282us/step - loss: 0.0340 - acc: 0.9902 - val_loss: 0.0739 - val_acc: 0.9811
Epoch 6/10
12707/12707 [==============================] - 4s 298us/step - loss: 0.0242 - acc: 0.9927 - val_loss: 0.0727 - val_acc: 0.9792
Epoch 7/10
12707/12707 [==============================] - 4s 287us/step - loss: 0.0203 - acc: 0.9928 - val_loss: 0.0768 - v

In [0]:
test_texts = np.array(test_data.text.apply(lambda x : remove_stopwords(x)))
sequences_test = tokenizer.texts_to_sequences(test_texts)
X_predict = pad_sequences(sequences_test, maxlen = X_train.shape[1])

In [0]:
prediction = keras_model.predict_classes(X_predict)

In [359]:
answers = transform_from_categorical(prediction, all_unique_intents)
answers

['AddToPlaylist', 'GetWeather', 'GetWeather', 'PlayMusic', 'GetWeather']